In [1]:
import pandas as pd

import uproot
import awkward as ak
import vector
vector.register_awkward()

from argparse import ArgumentParser

def get_jetconst_columns(jet_collection, n_const = 8):
    ''' Get constituent column names '''

    const_members = [
        'pt','eta','phi',
        # 'mass', # valid?
        'pdgId', # reco Id by PF algo, not full pdgID, but reduced variety for hadrons - has inefficiency! and not to be used directly
        'vz'
    ]

    const_columns = []

    for idau in range(n_const):
        const_columns += [jet_collection + "_dau%i_%s" %(idau,memb) for memb in const_members]

    return const_columns

def load_arrays(fname = "data/perfNano_TTbar_PU200_GenParts_More.root",
                jet_collection = "L1PuppiAK4Jets",
                n_const = 8,
                ):
    ''' Load arrays from ROOT file and return '''
    print("Reading arrays from %s" %fname)

    # open ROOT file
    tree = uproot.open(fname)["Events"]

    # get available jet collection names
    avail_jet_collections = [b[:-3] for b in tree.keys(filter_name="*Jets_pt")]

    ## check jet collections are present
    for colname in ["GenJets",jet_collection]:
        if colname not in avail_jet_collections:
            print(colname, " not available in ", fname)
            return 0

    ## get GenPart and GenJets
    gen_arrays = tree.arrays(library="ak", how="zip",
                             filter_name = "/(GenCands_|GenJets_)(pt|eta|phi|pdgId|status|partonFlavour|hadronFlavour)/"
                             )

    genparts = ak.with_name(gen_arrays.GenCands, "Momentum4D")
    genjets = ak.with_name(gen_arrays.GenJets, "Momentum4D")
    print("Loaded: %i events" %len(genjets))
    print("Loaded: %i genjets" %len(ak.flatten(genjets, axis = 1)))
    print("Loaded: %i genparts" %len(ak.flatten(genparts, axis = 1)))
    # get jets array for collection of interest
    if jet_collection == "GenJets":
        # no need to load anything extra, use genjets as reco jets
        l1jets = genjets
    else:
        jet_arrays = tree.arrays(library="ak", how="zip", filter_name = "/(%s_)(pt|eta|phi)/" %jet_collection )
        l1jets = ak.with_name(jet_arrays[jet_collection], "Momentum4D")
        print("Loaded: %i %s" %(len(ak.flatten(l1jets, axis = 1)), jet_collection))

    ## Load jet constituents
    const_columns = get_jetconst_columns(jet_collection, n_const)
    const_array = tree.arrays(library="ak", how="zip", filter_name = const_columns)
    jet_consts = const_array[jet_collection]

    return genparts, genjets, l1jets, jet_consts

def match_genParts(l1jets, genparts):
    ## The mapping of GenParts to be matched to
    gen_sel_ids = {
        ## light
        "g": (abs(genparts.pdgId) == 21) & (abs(genparts.status) == 71),
        "q": (abs(genparts.pdgId) < 5) & (abs(genparts.status) == 23),
        "b": (abs(genparts.pdgId) == 5) & (abs(genparts.status) == 23),
        ## heavy
        "t": (abs(genparts.pdgId) == 6) & (abs(genparts.status) == 22),
        "W": (abs(genparts.pdgId) == 24) & (abs(genparts.status) == 22),
        ## leptons
        "e": (abs(genparts.pdgId) == 11) & (abs(genparts.status) == 1),
        "mu": (abs(genparts.pdgId) == 13) & (abs(genparts.status) == 1),
        "tau": (abs(genparts.pdgId) == 15) & (abs(genparts.status) == 2),
    }


    ## matching to Gen Particles
    for part_label, part_sel in gen_sel_ids.items():

        jet_gen = ak.cartesian({"jets": l1jets, "gen": genparts[part_sel]}, nested=True)
        js, gs = ak.unzip(jet_gen)
        dR = js.deltaR(gs)

        l1jets["min_dR_" + part_label] = ak.min(dR, axis = -1)
        print(part_label, ak.sum(l1jets["min_dR_" + part_label] < 0.4))

    return l1jets

def match_genJets(l1jets, genjets):

    jet_gen = ak.cartesian({"jets": l1jets, "gen": genjets}, nested=True)
    js, gs = ak.unzip(jet_gen)
    dR = js.deltaR(gs)

    l1jets["min_dR_genjet"] = ak.min(dR, axis = -1)
    best_dR = ak.argmin(dR, axis=-1, keepdims=True)

    l1jets["partonFlavour"] = jet_gen[best_dR].gen.partonFlavour
    l1jets["hadronFlavour"] = jet_gen[best_dR].gen.hadronFlavour
    print (l1jets["hadronFlavour"])
    return l1jets

In [2]:
genparts, genjets, l1jets, jet_consts = load_arrays('perfNano.root','scPuppiCorrJets',64)

Reading arrays from perfNano.root
Loaded: 1000 events
Loaded: 6661 genjets
Loaded: 1713367 genparts
Loaded: 15052 scPuppiCorrJets


In [3]:
gen_sel_ids = {
        ## light
        "g": (abs(genparts.pdgId) == 21) & (abs(genparts.status) == 71),
        "q": (abs(genparts.pdgId) < 5) & (abs(genparts.status) == 23),
        "b": (abs(genparts.pdgId) == 5) & (abs(genparts.status) == 23),
        ## heavy
        "t": (abs(genparts.pdgId) == 6) & (abs(genparts.status) == 22),
        "W": (abs(genparts.pdgId) == 24) & (abs(genparts.status) == 22),
        ## leptons
        "e": (abs(genparts.pdgId) == 11) & (abs(genparts.status) == 1),
        "mu": (abs(genparts.pdgId) == 13) & (abs(genparts.status) == 1),
        "tau": (abs(genparts.pdgId) == 15) & (abs(genparts.status) == 2),
    }

In [4]:
jet_gen = ak.cartesian({"jets": l1jets, "gen": genparts[gen_sel_ids['g']]}, nested=True)
js, gs = ak.unzip(jet_gen)
dR = js.deltaR(gs)

In [24]:
dR

<Array [[6.12, 3.45, 3.8, ..., 2.38, 2.69], ...] type='1000 * var * float32'>

## Check what we lost from raw data

In [7]:
masses = ak.to_list(jet_consts.dau0_mass)
{x for l in masses for x in l}

{0.0, 0.004999999888241291, 0.10499999672174454, 0.12999999523162842, 0.5}

In [2]:
tree = uproot.open('perfNano.root')['Events']

In [4]:
print(tree['scPuppiCorrJets_dau1_vz'].title,
      tree['GenJets_dau0_pt'].title,
      tree['GenJets_pt'].title,
      sep='\n')

? numberOfDaughters() > 1 ? daughter(1).vertex.Z : -1
? numberOfDaughters() > 0 ? daughter(0).pt : -1
pt of jet


In [8]:
tree['scPuppiCorrJets_gendr'].arrays()

<Array [{scPuppiCorrJets_gendr: ..., ...}, ...] type='1000 * {scPuppiCorrJe...'>

In [14]:
{x for l in ak.to_list(tree.arrays(filter_name='GenCands_mass').GenCands_mass) for x in l}

{3.814697265625e-06,
 -1.789251655281987e-05,
 0.0,
 -1.2361031622276641e-05,
 8.529922524758149e-06,
 -3.0517578125e-05,
 -1.1757686479541007e-05,
 -7.152557373046875e-07,
 2.4722063244553283e-05,
 -2.15791860682657e-05,
 0.5,
 4.76837158203125e-07,
 1.52587890625e-05,
 -6.332993507385254e-08,
 2.0069799423217773,
 2.112299919128418,
 3.6865031719207764,
 2.010279893875122,
 2.4539663791656494,
 3.55588436126709,
 3.7750296592712402,
 4.800000190734863,
 3.644146203994751,
 3.686117649078369,
 4.902969837188721,
 4.880747318267822,
 5.325200080871582,
 5.41540002822876,
 5.279250144958496,
 5.366769790649414,
 5.279580116271973,
 5.960464477539063e-08,
 1.5,
 1.9073486328125e-06,
 9.5367431640625e-07,
 -2.384185791015625e-07,
 0.33000001311302185,
 0.5000001788139343,
 2.384185791015625e-07,
 0.3045954704284668,
 38.69084167480469,
 36.72555923461914,
 42.46591567993164,
 4.7960853066797426e-08,
 44.35298156738281,
 45.46894454956055,
 44.46318817138672,
 47.754058837890625,
 48.37515

In [12]:
tree.arrays(filter_name='GenCands_charge')

<Array [{GenCands_charge: [0, ...]}, ...] type='1000 * {GenCands_charge: va...'>

In [10]:
tree.arrays(filter_name='GenJets_mass')[0]

<Record {GenJets_mass: [15.4, ..., 6.72]} type='{GenJets_mass: var * float32}'>